ANN&DL Project Kaggle Competition - Image Classification
-------

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Environment settings
-----
Import necessary libraries and gpu memory growth enabling. Uncomment below "Allows to only as much GPU memory as needed" to activate Gpu memory growth. 

In [ ]:
import os
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)

In [ ]:
# ImageDataGenerator
# ------------------
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#directories used to handle the datasets
dataset_dir = os.path.join(cwd, 'Classification_Dataset')
training_dir = os.path.join(dataset_dir, 'training')
validation_dir = os.path.join(dataset_dir, 'validation')

Validation Split
------
20% of samples are moved into a new directory called validation.

In [ ]:
#Validation split in a reproducible way
#Done according to https://cs230-stanford.github.io/train-dev-test-split.html#have-a-reproducible-script


if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
    
for subdir, dirs, files in os.walk(training_dir):
    for class_dir in dirs:
        filenames = os.listdir(os.path.join(training_dir, class_dir))
        np.random.shuffle(filenames)
        split = int(0.8 * len(filenames))
        train_filenames = filenames[:split]
        valid_filenames = filenames[split:]
        if not os.path.exists(os.path.join(validation_dir, class_dir)):
            os.mkdir(os.path.join(validation_dir, class_dir))
        for file in valid_filenames:
            shutil.move(os.path.join(training_dir, class_dir, file), os.path.join(validation_dir, class_dir, file))
        

Data Augmentation
----
Creation of two ImageDataGenerator objects to scan Training and Validation directories. train_data_gen performs data augmentation on the training set.

In [ ]:
apply_data_augmentation = True

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=30,
                                        width_shift_range=25,
                                        height_shift_range=25,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='nearest',
                                        cval=0,
                                        rescale=1./255)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)

Data
----
- Batch size
- Image Shape
- Classes Indices
- Dataset Reading

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------

# Batch size
bs = 16

# img shape, competition dataset does not have regular shapes, must search a method to solve
img_h = 256
img_w = 256

num_classes = 20

decide_class_indices = True
if decide_class_indices:
    class_list = ['owl', # 0
               'galaxy', # 1
               'lightning', # 2
               'wine-bottle', # 3
               't-shirt', # 4
               'waterfall', # 5
               'sword', # 6
               'school-bus', # 7
               'calculator', # 8
               'sheet-music', # 9
               'airplanes', # 10
               'lightbulb', # 11
               'skyscraper', # 12
               'mountain-bike', # 13
               'fireworks', # 14
               'computer-monitor', # 15
               'bear', # 16
               'grand-piano', # 17
               'kangaroo', # 18
               'laptop'] # 19
else:
    classes = None

# Training

train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(img_h, img_w),
                                               batch_size=bs,
                                               classes=class_list,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  
# targets are directly converted into one-hot vectors

#for cycle to show training images

#from matplotlib import pyplot as plt
#%matplotlib inline
#n_array_images, n_array_labels = train_gen.next()

#for images in n_array_images[:8]:
#        plt.imshow(images)
#        plt.show()

# Validation,
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               target_size=(img_h, img_w),
                                               batch_size=bs, 
                                               classes=class_list,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

#for cycle to show validation images
#n_array_images, n_array_labels = valid_gen.next()
#print("validation Set")
#for images in n_array_images[:8]:
#        plt.imshow(images)
#        plt.show()

Dataset objects
----------------------

In [ ]:
# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))


# Repeat
# Without calling the repeat function the dataset 
# will be empty after consuming all the images
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

In [ ]:
train_gen.class_indices

Transfer Learning
-----
Import of Xception model.

In [ ]:
#transfer_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
transfer_model = tf.keras.applications.xception.Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_h, img_w, 3), pooling='max')
#transfer_model = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(img_h, img_w, 3), alpha=1.0, include_top=False, weights='imagenet', input_tensor=None, pooling=None)

In [ ]:
#Layers visualization
transfer_model.summary()
transfer_model.layers

First Training
----
Training of the fully connected layer of the model without fine tuning.

In [ ]:
for layer in transfer_model.layers:    
    layer.trainable = False    

model = tf.keras.Sequential()
model.add(transfer_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.2, noise_shape=None, seed=SEED))
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

Optimization params
-------------------

In [ ]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Experiments Directory Creation
----

In [ ]:
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'Xception'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    

Training
------
Model checkpoint creation.
Callbacks creation.
Early Stopping.
Model training with early stopping.

In [ ]:
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_class_{epoch:02d}.ckpt'),
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                   min_delta=0,
                                                   patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          callbacks=callbacks)

# How to visualize Tensorboard

# 1. tensorboard --logdir classification_experiments --port 6066     <- from terminal
# 2. localhost:6066   <- in your browser

Fine Tuning
----
Setting of the model section to train with fine tuning. 

In [ ]:
freeze_until = 126 # layer from which we want to fine-tune

for layer in model.layers[:freeze_until]:    
    layer.trainable = False
for layer in model.layers[freeze_until:]:
    layer.trainable = True

# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Re training
-----------
Second training of the model with fine tuning.

In [ ]:
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'Xception'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    

In [ ]:
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_fine_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                   min_delta=0,
                                                   patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          callbacks=callbacks)

Dataset Split Json Creator
---

In [ ]:
import json

directories = os.listdir(training_dir)

dataset_split = {}

dataset_split["training"] = dict((el,0) for el in directories)

for directory in directories:
    dataset_split["training"][directory] = os.listdir(os.path.join(training_dir,directory))
                                                    
directories = os.listdir(validation_dir)

dataset_split["validation"] = dict((el,0) for el in directories)

for directory in directories:
    dataset_split["validation"][directory] = os.listdir(os.path.join(validation_dir,directory))
    
with open('dataset_split.json', 'w') as fp:
     json.dump(dataset_split, fp)

Dataset Merge
----
Merge of validaton and training directories to restore the original dataset.

In [ ]:
for subdir, dirs, files in os.walk(validation_dir):
    for file in files:
        class_dir = os.path.basename(subdir)
        shutil.move(os.path.join(subdir, file), os.path.join(training_dir, class_dir, file))
        
shutil.rmtree(validation_dir)

In [ ]:
def create_csv(results, results_dir=cwd):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

Prediction
---
Creation of csv file with the prediction of the model on the test dataset.

In [ ]:
from PIL import Image

# Test
test_dir = os.path.join(dataset_dir, 'test')
image_filenames = next(os.walk(test_dir))[2]

results = {}
for image_name in image_filenames:
    file_path = os.path.join(test_dir, image_name)
    img = Image.open(file_path).convert('RGB')
    img = img.resize((img_w, img_h))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, 0)
    out_softmax = model.predict(x=img_array / 255.)
    prediction = tf.argmax(out_softmax, -1)
    results[image_name] = prediction[0].numpy()
    
create_csv(results)
results